In [32]:
import torch
from abc import ABC, abstractmethod
import numpy as np

In [33]:
def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width

In [34]:
class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


In [35]:
def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)

In [36]:
class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)

In [44]:
def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)
    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)

In [161]:
class Conv2dMatrixV2(ABCConv2d):
    # Функция преобразования кернела в нужный формат.
    def _convert_kernel(self):
        matrix_zeros = np.zeros((self.kernel.size()[0],self.kernel.size()[1] * self.kernel.size()[2]*self.kernel.size()[3]))
        j, k = 0, 0
        for image in self.kernel[0].numpy():
          for i in range(len(image)):
            matrix_zeros[:, k:k+self.kernel.size()[2]] = image[i]
            k += self.kernel.size()[2]
          j+=1
        converted_kernel = torch.from_numpy(matrix_zeros).float()# Реализуйте преобразование кернела.
        return converted_kernel

    # Функция преобразования входа в нужный формат.
    def _convert_input(self, torch_input, output_height, output_width):
        matrix_zeros = np.zeros((torch_input.size()[1] *self.kernel.size()[2]*self.kernel.size()[3], torch_input.size()[0] ))
        for i in range(matrix_zeros.shape[1]):
          j = 0
          for core in torch_input[i].numpy():
            core_reshape = core.reshape(1, core.shape[0]*core.shape[1])
            k = 0
            for g in range(self.kernel.size()[2]):
              matrix_zeros[j:j+self.kernel.size()[2], i] = core_reshape[:,k:k+self.kernel.size()[2]] # np.flip(core_reshape, 1)
              j += self.kernel.size()[2]
              k += self.kernel.size()[2]+1
        converted_input = torch.from_numpy(matrix_zeros).float() # Реализуйте преобразование входа.
        return converted_input

    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        converted_kernel = self._convert_kernel()
        converted_input = self._convert_input(torch_input, output_height, output_width)

        conv2d_out_alternative_matrix_v2 = converted_kernel @ converted_input
        
        return conv2d_out_alternative_matrix_v2.transpose(0, 1).view(torch_input.shape[0], self.out_channels, output_height, output_width).transpose(1, 3).transpose(2, 3)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
print(test_conv2d_layer(Conv2dMatrixV2))

test tensor([[[[ 5252.]]],


        [[[12596.]]]]) tensor([[[[ 5252.]]],


        [[[12596.]]]], grad_fn=<MkldnnConvolutionBackward>)
True
